In [27]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import glob
import os
import math
from os import listdir
import data_visualization as dv
import pickle
import scipy 
from scipy.stats import norm
from scipy import stats
from tqdm import tqdm 
'''
This script pulls
1. Decision array of agent and player
2. Decision time of agent and player
   - Possibly calculate this with velocity in the future
3. Control things
   - Reaction times, interval target reach times, coincidence target reach times
   
   
*****v2 GOES BACK TO THE REACTION WITH THE CIRCLES INSTEAD OF THE AGENT****
'''

'\nThis script pulls\n1. Decision array of agent and player\n2. Decision time of agent and player\n   - Possibly calculate this with velocity in the future\n3. Control things\n   - Reaction times, interval target reach times, coincidence target reach times\n   \n   \n*****v2 GOES BACK TO THE REACTION WITH THE CIRCLES INSTEAD OF THE AGENT****\n'

In [28]:
# Fields pull and pull list
os.chdir('D:\Subject_Data\Seth_MatchPennies_Agent_Exp1')
PATH = os.getcwd()
analysis_pull_list = []
analysis_pull_list_control = []
analysis_pull_list_task = []
fields_pull = []
with open(PATH+"\\Analysis_Pull_List.txt", "r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
with open(PATH+"\\Analysis_Pull_List_Control.txt", "r") as pull_file:
    analysis_pull_list_control = pull_file.read().splitlines()
with open(PATH+"\\Analysis_Pull_List_Task.txt", "r") as pull_file:
    analysis_pull_list_task = pull_file.read().splitlines()
with open(PATH+"\\Fields_Pull.txt", "r") as fields_pull:
    fields_pull = fields_pull.read().splitlines()
    
num_subjects = len(analysis_pull_list_control)
task_name = 'Seth_MatchPennies_Agent_Exp1'



Target information

- Row 0 = Start Target

- Row 1 = Left screen, right target x pos (left target is 2*startx - right targetx)

- Dim 1 = Radius

- Dim 2 = Thickness of the circle edge (don't know if this matters)


Dataframe is in centimeters, so need to divide everything by 100

In [29]:
file = PATH+'\\Sub1_Task\\Sub1_TaskTarget_Table.csv'
df = pd.read_csv(file)
df["X"] = df["X"]/100
df["Y"] = df["Y"]/100
df['Dim 1'] = df['Dim 1']/100 # Target table is in centimeters, I guess this doesn't matter but it makes me feel better
df['Dim 2'] = df['Dim 2']/100
# Target information for Right Hand (keeping this because the positions of target 3 and 4 are based on target 1 and start 1)
startx = df.loc[0]['X']
starty = df.loc[0]['Y']
start_radius = df.loc[0]['Dim 1'] 

target1x = df.loc[1]['X']
target1y = df.loc[1]['Y']
target1_radius = df.loc[1]['Dim 1']

target2x = 2*startx - target1x
target2y = target1y
target2_radius = target1_radius

# Timing target
timing_targetx = startx
timing_targety = target1y
timing_target_pos = np.sqrt(timing_targetx**2 + timing_targety**2)
timing_target_radius = target1_radius   

Control Experiment Data

In [30]:
#%% Get reaction time data
reaction_trial_time = 5000
reaction_trials = 50
player_reaction_decision_array = np.empty((num_subjects, reaction_trials))
reaction_trial_start = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_movement_time = np.zeros((num_subjects, reaction_trials))*np.nan
reaction_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_time = np.zeros((num_subjects, reaction_trials))*np.nan
agent_reaction_decision_time = np.zeros((num_subjects, reaction_trials))*np.nan
player_reaction_reach_time = np.zeros((num_subjects, reaction_trials))*np.nan
reaction_data = np.zeros((num_subjects, reaction_trials, reaction_trial_time, 2))*np.nan


coincidence_trial_time = 2500
coincidence_trials = 50
coincidence_trial_start = np.zeros((num_subjects, coincidence_trials))*np.nan
coincidence_reach_time = np.zeros((num_subjects, coincidence_trials))*np.nan
coincidence_data = np.zeros((num_subjects, coincidence_trials, coincidence_trial_time, 2))*np.nan

interval_trial_time = 2500
interval_trials = 50
interval_trial_start = np.zeros((num_subjects, interval_trials))*np.nan
interval_reach_time =  np.zeros((num_subjects, interval_trials))*np.nan
interval_data = np.zeros((num_subjects, interval_trials, interval_trial_time, 2))*np.nan

control_trials = interval_trials + coincidence_trials + reaction_trials
###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_control[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    control_trial_table = pd.read_csv(file)
    block_number = 1
    tp_num = 1
    for x in tqdm((range(control_trials))):
        block_number = control_trial_table.iloc[x]['Block_Row']
        tp_num = control_trial_table.iloc[x]['TP_Row']
        block_trial_num = control_trial_table.iloc[x]['Block_Step']
        filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        j = tp_num - 1 # Block number
        k = block_trial_num - 1 # Trial number in that block
        data = pd.read_csv(filename, low_memory=False)
        # if 'Agent_Initial_Time' in data.columns:
        #     data = data.rename(columns = {'Agent_Inital_Time':'Agent_Decision_Time'}, inplace=True)
        # f = data['Agent_Decision_Time']
        start_time = int(data[data['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        # Get Interval Timing Data
        if tp_num == 1:
            end_time = start_time + interval_trial_time
            interval_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                interval_reach_time[i,k] = q[0]
        # Get Coincidence Timing Data
        if tp_num == 2:
            end_time = start_time + coincidence_trial_time
            coincidence_trial_start[i,k] = start_time # Store start time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-timing_targetx)**2 + (lhy-timing_targety)**2) < timing_target_radius) # THIS NEEDS TO BE THE CENTER TARGET... x poisition should be the start
            if np.size(q)>0:
                coincidence_reach_time[i,k] = q[0]
        # Get Reaction Time Data
        if tp_num == 3:
            end_time = start_time + reaction_trial_time
            reaction_trial_start[i,k] = start_time
            lhx = np.array(data.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(data.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
            r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
            s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*2) # Multiplied by 2 so people don't accidentally drift outside start
            if np.size(s)>0: 
                player_reaction_time[i,k] = s[0] # lhx and lhy are already constrained from start_time to end_time, so as soon as they leave the start, that's their reaction time
            if np.size(q)>0: #if LH enters right target
                player_reaction_decision_array[i,k] = 1
                player_reaction_movement_time[i,k] = q[0] - s[0]
                player_reaction_reach_time[i,k] = q[0]
            elif np.size(r)>0: #if LH enters left target
                player_reaction_decision_array[i,k] = -1   
                player_reaction_movement_time[i,k] = r[0] - s[0]
                player_reaction_reach_time[i,k] = r[0]
                
                
            # #plot path for Reactions
            if False:
                for i in range(1):
                    plot_end_time = int(start_time + player_reaction_reach_time[i,k])
                    lhx_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                    lhy_new = np.array(data.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                    plt.figure(dpi=300)
                    circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                    circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                    startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                    fig, ax = plt.subplots()
                    ax.add_patch(circleR)
                    ax.add_patch(circleL)
                    ax.add_patch(startCirc)
                    plt.plot(lhx_new,lhy_new) 
                    plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])

                    plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                    plt.show()

           

0 Sub1_Control


100%|██████████| 150/150 [00:02<00:00, 65.54it/s]


1 Sub2_Control


100%|██████████| 150/150 [00:01<00:00, 84.39it/s]


2 Sub3_Control


...

100%|██████████| 150/150 [00:01<00:00, 93.99it/s]


10 Sub11_Control


100%|██████████| 150/150 [00:02<00:00, 73.43it/s]


11 Sub12_Control


100%|██████████| 150/150 [00:01<00:00, 91.46it/s]


In [31]:
analysis_pull_list = []
with open(PATH+"\\Analysis_Pull_List.txt","r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(data_path)
    # Reaction time 
    pickle.dump(reaction_trial_start[i,:], open(data_path + f'{subname}_reaction_trial_start.pkl', 'wb'))
    print(player_reaction_time[i,:])
    pickle.dump(player_reaction_time[i,:], open(data_path + f'{subname}_player_reaction_time.pkl', 'wb'))
    pickle.dump(player_reaction_movement_time[i,:], open(data_path + f'{subname}_player_reaction_movement_time.pkl', 'wb'))
    pickle.dump(player_reaction_decision_array[i,:], open(data_path + f'{subname}_player_reaction_decision_array.pkl', 'wb'))
    pickle.dump(agent_reaction_decision_time[i,:], open(data_path + f'{subname}_agent_reaction_decision_time.pkl', 'wb'))
    pickle.dump(reaction_trial_start[i,:], open(data_path + f'{subname}_reaction_trial_start.pkl', 'wb'))
    pickle.dump(interval_trial_start[i,:], open(data_path + f'{subname}_interval_trial_start.pkl', 'wb'))
    pickle.dump(interval_reach_time[i,:], open(data_path + f'{subname}_interval_reach_time.pkl', 'wb'))
    pickle.dump(coincidence_trial_start[i,:], open(data_path + f'{subname}_coincidence_trial_start.pkl', 'wb'))
    pickle.dump(coincidence_reach_time[i,:], open(data_path + f'{subname}_coincidence_reach_time.pkl', 'wb'))


D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub1\
[570. 417. 289. 319. 315. 346. 333. 315. 356. 457. 243. 247. 258. 288.
 307. 293. 269. 306. 275. 292. 251. 319. 268. 295. 326. 261. 314. 258.
 303. 294. 275. 328. 276. 286. 264. 249. 270. 271. 257. 277. 239. 312.
 261. 255. 266. 279. 320. 301. 260. 285.]
D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub2\
[497. 329. 334. 259. 300. 297. 292. 302. 299. 327. 325. 305. 338. 286.
 275. 293. 285. 344. 346. 313. 271. 315. 318. 298. 283. 300. 330. 299.
 349. 314. 338. 337. 335. 270. 294. 314. 309. 324. 308. 302. 308. 325.
 308. 284. 309. 270. 291. 350. 316. 316.]
D:\Subject_Data\Seth_MatchPennies_Agent_Exp1\Subjects_Analyzed\Sub3\
[387. 266. 323. 389. 351. 327. 304. 343. 309. 362. 309. 343. 272. 263.
 292. 323. 264. 328. 313. 338. 378. 346. 283. 308. 320. 294. 313. 344.
 451. 316. 325. 326. 390. 319. 410. 359. 352. 390. 293. 294. 327. 380.
 280. 290. 352. 282. 372. 353. 326. 394.]
D:\Subject_Data\Seth_MatchPe

Task Data 

In [32]:
#%% Get task time task_df
# Get trials, blocks, trial_time from trial_table
path1 = PATH+'\\'+'Sub1_Task'
task_df = pd.read_csv(path1+f'\\Sub1_TaskTrial_Table.csv')
task_df = task_df.loc[task_df['Condition type']==3] # Only get the task condition 
num_trials = int(task_df.iloc[-1]['Block_Step']) # number of trials in each block
num_blocks = int(task_df.iloc[-1]['Block_Row']/2)
tot_trials = int(num_trials*num_blocks)
trial_time = int(task_df.iloc[0]['Condition time'])
task_df_columns = len(fields_pull)
# Need to be kept outside subject for-loop
task_trial_end = np.zeros([num_subjects, num_blocks, num_trials, 2])*np.nan
task_trial_start = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
filename_storage = np.empty((num_subjects, num_blocks, num_trials), dtype=object)

task_data = np.zeros((num_subjects, num_blocks, num_trials, trial_time+500, task_df_columns))*np.nan
win_check_R = np.zeros((num_subjects, num_blocks))
win_check_L = np.zeros((num_subjects, num_blocks))
num_miss_check_R = np.zeros((num_subjects, num_blocks))
num_miss_check_L = np.zeros((num_subjects, num_blocks))
#########
player_task_decision_array = np.empty((num_subjects, num_blocks, num_trials))
player_task_reach_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_task_decision_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_task_movement_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
agent_task_decision_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_task_reach_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_task_decision_array = np.zeros((num_subjects, num_blocks, num_trials))

###-------------------------------------------------------------------------------------------------------
for i in range(num_subjects):
    sub_name = analysis_pull_list_task[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    trial_table = pd.read_csv(file)
    # Splt trial table into task and washout based on condition type
    task_trial_table = trial_table[trial_table['Condition type']==3]
    block_number = 1
    tp_num = 1
    for x in tqdm((range(tot_trials))):
        block_number = task_trial_table.iloc[x]['Block_Row']
        tp_num = task_trial_table.iloc[x]['TP_Row']
        block_trial_num = task_trial_table.iloc[x]['Block_Step']
        j = tp_num - task_trial_table['TP_Row'].min()  # Block number
        k = block_trial_num - 1 # Trial number in that block
        
        filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
        task_df = pd.read_csv(filename, low_memory=False)
        start_time = int(task_df[task_df['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
        end_time = start_time + trial_time + 500 # Add 500 to see when they got there when trial ended
        data_task_df = task_df.drop(columns ='Event_Codes') # Drop event codes cuz it's not number column and can't be an array 
        task_data[i,j,k,:,:] = np.array(data_task_df.iloc[start_time:end_time])

        task_trial_start[i,j,k] = start_time
        lhx = np.array(task_df.iloc[start_time:end_time]['Left_HandX'])
        lhy = np.array(task_df.iloc[start_time:end_time]['Left_HandY'])
        q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
        r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
        s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*2) # Multiplied by 2 so people don't accidentally drift outside start
        
        #print(vel_check)
        agent_task_decision_time[i,j,k] = task_df.iloc[start_time+1]['Agent_Decision_Time'] 
        agent_task_reach_time[i,j,k] = agent_task_decision_time[i,j,k] + 150  
        agent_task_decision_array[i,j,k] = task_df.iloc[start_time+1]['Agent_Target_Selection']
        if np.size(s)>0: 
            vel_check = task_df.iloc[s[0]]['Left_HandYVel']
            player_task_decision_time[i,j,k] = s[0]
            
        if np.size(q)>0: #if LH enters right target
            player_task_decision_array[i,j,k] = 1
            player_task_movement_time[i,j,k] = q[0] - s[0]
            player_task_reach_time[i,j,k] = q[0]
        elif np.size(r)>0: #if LH enters left target
            player_task_decision_array[i,j,k] = -1   
            player_task_movement_time[i,j,k] = r[0] - s[0]
            player_task_reach_time[i,j,k] = r[0]
        if player_task_reach_time[i,j,k] > 1500 or np.isnan(player_task_reach_time[i,j,k]):
            player_task_decision_array[i,j,k] = 0
        if agent_task_reach_time[i,j,k]>1500:
            agent_task_decision_array[i,j,k] = 0    
        #plot path for tasks
        if False:
            for i in range(1):
                plot_end_time = int(start_time + player_task_reach_time[i,j,k])
                lhx_new = np.array(task_df.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                lhy_new = np.array(task_df.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                plt.figure(dpi=300)
                circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                fig, ax = plt.subplots()
                ax.add_patch(circleR)
                ax.add_patch(circleL)
                ax.add_patch(startCirc)
                plt.plot(lhx_new,lhy_new) 
                plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])

                plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                plt.show()

           

0 Sub1_Task


100%|██████████| 480/480 [00:07<00:00, 61.78it/s]


1 Sub2_Task


100%|██████████| 480/480 [00:08<00:00, 55.47it/s]


2 Sub3_Task


...

100%|██████████| 480/480 [00:09<00:00, 52.43it/s]


10 Sub11_Task


100%|██████████| 480/480 [00:07<00:00, 64.49it/s]


11 Sub12_Task


100%|██████████| 480/480 [00:12<00:00, 38.26it/s]


Pickle Task Data

In [33]:
# Pickle
analysis_pull_list = []
with open(PATH+"\\Analysis_Pull_List.txt","r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    pickle.dump(task_data[i,:,:,:,:], open(data_path + f'{subname}_task_data.pkl', 'wb'))
    pickle.dump(player_task_decision_time[i,:,:], open(data_path + f'{subname}_player_task_decision_time.pkl', 'wb'))
    pickle.dump(player_task_decision_array[i,:,:], open(data_path + f'{subname}_player_task_decision_array.pkl', 'wb'))
    pickle.dump(player_task_movement_time[i,:,:], open(data_path + f'{subname}_player_task_movement_time.pkl', 'wb'))
    pickle.dump(player_task_reach_time[i,:,:], open(data_path + f'{subname}_player_task_reach_time.pkl', 'wb'))
    pickle.dump(agent_task_decision_time[i,:,:], open(data_path + f'{subname}_agent_task_decision_time.pkl', 'wb'))
    pickle.dump(agent_task_decision_array[i,:,:], open(data_path + f'{subname}_agent_task_decision_array.pkl', 'wb'))
    pickle.dump(agent_task_reach_time[i,:,:], open(data_path + f'{subname}_agent_task_reach_time.pkl', 'wb'))

Sub1
Sub2
Sub3
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12


Washout Data

In [34]:
#%% Get washout time washout_df
# Get trials, blocks, trial_time from trial_table
path1 = PATH+'\\'+'Sub1_Task'
trial_table = pd.read_csv(path1+'\\Sub1_TaskTrial_Table.csv')
block_table = pd.read_csv(path1+'\\Sub1_TaskBlock_Table.csv')
washout_trial_table = trial_table[trial_table['Condition type']==4] # Only get the washout condition 
num_trials = int(washout_trial_table.iloc[-1]['Block_Step']) # number of trials in each block
num_blocks = pd.DataFrame(block_table[block_table['TP_LIST']==12]['TP_LIST'].value_counts()).iloc[0]['TP_LIST'] # Counts how many times TP_LIST is 12 (washout tp), and then gets that number with .iloc[0]['TP_LIST]
tot_trials = int(num_trials*num_blocks)
trial_time = int(washout_trial_table.iloc[0]['Condition time'])
washout_columns = len(fields_pull)
# Need to be kept outside subject for-loop
washout_trial_end = np.zeros([num_subjects, num_blocks, num_trials, 2])*np.nan
washout_trial_start = np.zeros((num_subjects, num_blocks, num_trials))*np.nan
filename_storage = np.empty((num_subjects, num_blocks, num_trials), dtype=object)

washout_data = np.zeros((num_subjects, num_blocks, num_trials, trial_time+500, washout_columns))*np.nan
win_check_R = np.zeros((num_subjects, num_blocks))
win_check_L = np.zeros((num_subjects, num_blocks))
num_miss_check_R = np.zeros((num_subjects, num_blocks))
num_miss_check_L = np.zeros((num_subjects, num_blocks))
#########
player_washout_decision_array = np.empty((num_subjects, num_blocks, num_trials))
player_washout_reach_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_washout_decision_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
player_washout_movement_time = np.zeros([num_subjects, num_blocks, num_trials])*np.nan
agent_washout_decision_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_washout_reach_time = np.zeros((num_subjects, num_blocks, num_trials))
agent_washout_decision_array = np.zeros((num_subjects, num_blocks, num_trials))

###-------------------------------------------------------------------------------------------------------
for i in (range(num_subjects)):
    sub_name = analysis_pull_list_task[i]
    print(i, sub_name)
    path1 = PATH+'\\'+sub_name
    file = path1+f'\\{sub_name}Trial_Table.csv'
    trial_table = pd.read_csv(file)
    # Splt trial table into washout and washout based on condition type
    washout_trial_table = trial_table[trial_table['Condition type']==4]
    x = -1
    for j in (range(num_blocks)):
        for k in tqdm(range(num_trials)):
            x += 1
            block_number = washout_trial_table.iloc[x]['Block_Row']
            tp_num = washout_trial_table.iloc[x]['TP_Row']
            block_trial_num = washout_trial_table.iloc[x]['Block_Step']
            filename = PATH+f"\\{sub_name}\\{task_name}_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
            washout_df = pd.read_csv(filename, low_memory=False)
            start_time = int(washout_df[washout_df['Event_Codes']=='E_SOUND_SIGNAL'].index[0])
            end_time = start_time + trial_time + 500 # Add 500 to see when they got there when trial ended
            data_washout_df = washout_df.drop(columns ='Event_Codes') # Drop event codes cuz it's not number column and can't be an array 
            washout_data[i,j,k,:,:] = np.array(data_washout_df.iloc[start_time:end_time])

            washout_trial_start[i,j,k] = start_time
            lhx = np.array(washout_df.iloc[start_time:end_time]['Left_HandX'])
            lhy = np.array(washout_df.iloc[start_time:end_time]['Left_HandY'])
            q = np.argwhere(np.sqrt((lhx-target1x)**2 + (lhy-target1y)**2) < target1_radius)
            r = np.argwhere(np.sqrt((lhx-target2x)**2 + (lhy-target2y)**2) < target2_radius) 
            s = np.argwhere(np.sqrt((lhx-startx)**2 + (lhy-starty)**2) > start_radius*2) # Multiplied by 2 so people don't accidentally drift outside start
            
            #print(vel_check)
            agent_washout_decision_time[i,j,k] = washout_df.iloc[start_time+1]['Agent_Decision_Time']   
            agent_washout_decision_array[i,j,k] = washout_df.iloc[start_time+1]['Agent_Target_Selection']
            if np.size(s)>0: 
                vel_check = washout_df.iloc[s[0]]['Left_HandYVel']
                player_washout_decision_time[i,j,k] = s[0]
                
            if np.size(q)>0: #if LH enters right target
                player_washout_decision_array[i,j,k] = 1
                player_washout_movement_time[i,j,k] = q[0] - s[0]
                player_washout_reach_time[i,j,k] = q[0]
            elif np.size(r)>0: #if LH enters left target
                player_washout_decision_array[i,j,k] = -1   
                player_washout_movement_time[i,j,k] = r[0] - s[0]
                player_washout_reach_time[i,j,k] = r[0]
            if player_washout_reach_time[i,j,k] > 1500 or np.isnan(player_washout_reach_time[i,j,k]):
                player_washout_decision_array[i,j,k] = 0
            
        # #plot path for washouts
        if False:
            for i in range(1):
                plot_end_time = int(start_time + player_washout_reach_time[i,j,k])
                lhx_new = np.array(washout_df.iloc[start_time:int(plot_end_time)]['Left_HandX'])
                lhy_new = np.array(washout_df.iloc[start_time:int(plot_end_time)]['Left_HandY'])
                plt.figure(dpi=300)
                circleR = plt.Circle((target1x,target1y), target1_radius, color = 'r', fill = False)
                circleL = plt.Circle((target2x,target2y), target2_radius, color = 'r', fill = False)
                startCirc = plt.Circle((startx,starty), start_radius, color = 'r', fill = False)
                fig, ax = plt.subplots()
                ax.add_patch(circleR)
                ax.add_patch(circleL)
                ax.add_patch(startCirc)
                plt.plot(lhx_new,lhy_new) 
                plt.scatter(lhx_new[int(s[0])], lhy_new[int(s[0])])

                plt.title("w =%1.0f " %i + "x=%1.0f "%j + "c=%1.0f"%k + 'vely=%1.5f'%vel_check)
                plt.show()

           

0 Sub1_Task


100%|██████████| 25/25 [00:00<00:00, 63.53it/s]


1 Sub2_Task


100%|██████████| 25/25 [00:00<00:00, 54.96it/s]


2 Sub3_Task


...

100%|██████████| 25/25 [00:00<00:00, 53.81it/s]


10 Sub11_Task


100%|██████████| 25/25 [00:00<00:00, 65.84it/s]


11 Sub12_Task


100%|██████████| 25/25 [00:00<00:00, 39.53it/s]


Pickle Washout

In [35]:
# Pickle
analysis_pull_list = []
with open(PATH+"\\Analysis_Pull_List.txt","r") as pull_file:
    analysis_pull_list = pull_file.read().splitlines()
i=-1
for subname in analysis_pull_list:
    i+=1
    data_path = PATH+f'\\Subjects_Analyzed\\{subname}\\'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    print(subname)
    pickle.dump(washout_data[i,:,:,:,:], open(data_path + f'{subname}_washout_data.pkl', 'wb'))
    pickle.dump(player_washout_decision_time[i,:], open(data_path + f'{subname}_player_washout_decision_time.pkl', 'wb'))
    pickle.dump(player_washout_decision_array[i,:], open(data_path + f'{subname}_player_washout_decision_array.pkl', 'wb'))
    pickle.dump(player_washout_movement_time[i,:], open(data_path + f'{subname}_player_washout_movement_time.pkl', 'wb'))
    pickle.dump(player_washout_reach_time[i,:], open(data_path + f'{subname}_player_washout_reach_time.pkl', 'wb'))
    pickle.dump(agent_washout_decision_time[i,:], open(data_path + f'{subname}_agent_washout_decision_time.pkl', 'wb'))
    pickle.dump(agent_washout_decision_array[i,:], open(data_path + f'{subname}_agent_washout_decision_array.pkl', 'wb'))
    pickle.dump(agent_washout_reach_time[i,:], open(data_path + f'{subname}_agent_washout_reach_time.pkl', 'wb'))

Sub1
Sub2
Sub3
Sub4
Sub5
Sub6
Sub7
Sub8
Sub9
Sub10
Sub11
Sub12


Keep Order That Participants Did Condition

In [38]:
agent_task_decision_array[0]

array([[-1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,
        -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
         1., -1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,  1.,
         1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
         1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,
         1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
        -1., -1.],
       [-1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,
        -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
        -1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,
         0.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,
         1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,
         1.,  1.],
       [ 1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,  1.,
        -1., 

Use event codes to get wins and indecisions

In [37]:
# # Use event codes to find number of wins and indecisions
# #%% Extract Hand Position data
# for w in range(num_subjects): #len(pull_list)):
#     print(w)
#     sub_name = analysis_pull_list_task[w]
#     print(sub_name)
#     path1 = PATH+'\\'+sub_name
#     file = path1+f'\\{sub_name}Trial_Table.csv'
#     task_trial_table = pd.read_csv(file)
#     task_trial_table = task_trial_table.loc[task_trial_table['TP_Row']>=3]
#     for i in tqdm((range(tot_trials))):
#         block_number = task_trial_table.iloc[i]['Block_Row']
#         tp_num = task_trial_table.iloc[i]['TP_Row']
#         block_trial_num = task_trial_table.iloc[i]['Block_Step']
#         x = tp_num - 3 #used for indexing, subtraction number needs to change to 1 if I start with TP1 
#         c = block_trial_num - 1 #used for indexing, block trial number starts with 1
#         filename = PATH+f"\\{sub_name}\\Seth_MatchPennies_Agent_Pilot_v3_{sub_name}_C{block_number}_TP{tp_num}_T{block_trial_num}.csv"
#         filename_storage[w,x,c] = filename
#         data = pd.read_csv(filename, low_memory=False) #read in data
#         #Create start time and max end time array
#         score_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         trial_start[w,x,c]= int(data[data['Event_Codes']==list_event_codes[0]].index[0]) #if event codes column is equal to the list event codes, give the index (time)
#         trial_start = trial_start.astype(int)
#         end_time = trial_start[w,x,c]+2000
#         agent_target_select_index = int(data[data['Event_Codes']=='E_END_TRIAL'].index[0]) 
#         # Get Agent mean reach time... data starts at next index from E_SOUNG_SIGNAL aka trial_start
#         agent_decision_time[w,x,c] = data.iloc[trial_start[w,x,c]+1]['Agent_Reach_Time'] #CALLED AGENT REACH TIME BUT IT'S NOW DECISION TIME IN V3
#         agent_decision_array[w,x,c] = data.iloc[agent_target_select_index]['Agent_decision_array']
#         agent_indecision_index = np.argwhere(agent_decision_time+150>1500)
#         for i,j,k in agent_indecision_index:
#             agent_decision_array[i,j,k] = 0
#         data = np.array(data)
#         event_codes[w,x,c,:] = data[trial_start[w,x,c]:end_time,4] # create event codes array
#         hand_data = data[:,0:4] # data is only columns 0 thru 3 (RHX,RHY,LHX,LHY)
#         hand_data = hand_data.astype(float)
#         hand_pos_data[w,x,c,:,:] = hand_data[trial_start[w,x,c]:end_time,:]
        
#         # If you want the scores
#         # scoreA = data[score_index, 4]
#         # scoreB = data[score_index, 5]
#         # if scoreA == 1:
#         #     win_check_R[w,x]+=1
#         # if scoreB == 1:
#         #     win_check_L[w,x]+=1
#         # print(filename)
#         # print(data[trial_start[w,x,c]:end_time,:])
#         #------------------------------------------------------------------
           
#         # LEFT HAND
#         lhx = hand_pos_data[w,x,c,:,2]
#         lhy = hand_pos_data[w,x,c,:,3]
#         q = np.argwhere(np.sqrt((lhx-target3x)**2 + (lhy-target3y)**2) < target3_radius/2)
#         r = np.argwhere(np.sqrt((lhx-target4x)**2 + (lhy-target4y)**2) < target4_radius/2) 
#         s = np.argwhere(np.sqrt((lhx-start2x)**2 + (lhy-start2y)**2) > start2_radius/2)
#         if np.size(s)>0: 
#             player_decision_time[w,x,c] = s[0]
#         if np.size(q)>0: #if LH enters right target
#             player_decision_array[w,x,c] = 1
#             player_reach_time[w,x,c] = q[0]
#         elif np.size(r)>0: #if LH enters left target
#             player_decision_array[w,x,c] = -1   
#             player_reach_time[w,x,c] = r[0]
            
#         if player_reach_time[w,x,c] > 1500 or np.isnan(player_reach_time[w,x,c]):
#             player_decision_array[w,x,c] = 0